In [2]:
from args import argument_parser
from data_manager import BlockFrameDataset
from data_split import split_data
from model import CustomModel
from AverageMeter import AverageMeter

from tqdm.notebook import tqdm

import numpy as np

import warnings

import torch
from torch.utils.data import DataLoader
from torch import optim
import torch.nn.functional as F

In [3]:
# Code For GPU Selection
has_cuda = True if torch.has_cuda else False
has_mps = True if torch.has_mps else False

device = torch.device('cpu')
if has_cuda :
    device = torch.device('cuda')
    print("Using Cuda GPU")

elif has_mps :
    device = torch.device('mps')
    print("Using MPS GPU")

else :
    warnings.warn("Using SPU , GPU is Recommended")

print("Initializing Dataloader")

#Split Dataset
train_data,val_data,test_data = split_data("./dataset")

val_dataset = BlockFrameDataset("./dataset",val_data,block_size=8)
train_dataset = BlockFrameDataset("./dataset",train_data,block_size=8)
test_dataset = BlockFrameDataset("./dataset",test_data,block_size=8)

val_loader = DataLoader(val_dataset,shuffle=True,batch_size=20,num_workers=1, pin_memory=True)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=20,num_workers=1, pin_memory=True)
test_loader = DataLoader(test_dataset,shuffle=True,batch_size=20,num_workers=1, pin_memory=True)

# Procuring the pretrained model

model,processor = CustomModel()

learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-8)

Using MPS GPU
Initializing Dataloader


In [4]:
def train(model, processor, data_loader, optimizer, device):
    # meter
    loss_meter = AverageMeter()
    # switch to train mode
    model.train()
    model.to(device)
    tk = tqdm(data_loader, total=int(len(data_loader)), desc='Training', unit='frames', leave=False)
    for batch_idx, data in enumerate(tk):
        # fetch the data
        frame, label = data[0], data[1]
        frame = torch.squeeze(frame)
        label = label
        # after fetching the data, transfer the model to the 
        # required device, in this example the device is gpu
        # transfer to gpu can also be done by 
        frame, label = frame.to(device), label.to(device)
        # compute the forward pass
        output = model(frame)
        logits = output.logits
        # predicted_class_idx = logits.argmax(-1).item()


        # print("Frame : ",frame[0].shape," Label: ",label," Output: ",output," Predicted : ",predicted_class_idx)
        # compute the loss function
        loss_this = F.cross_entropy(logits, label)
        # initialize the optimizer
        optimizer.zero_grad()
        # compute the backward pass
        loss_this.backward()
        # update the parameters
        optimizer.step()
        # update the loss meter
        loss_meter.update(loss_this.item(), label.shape[0])
        tk.set_postfix({"loss": loss_meter.avg})
    print('Train: Average loss: {:.4f}\n'.format(loss_meter.avg))

In [5]:
train(model,processor,train_loader,optimizer,device)

Training:   0%|          | 0/135 [00:00<?, ?frames/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
Train: Average loss: 0.5880



In [6]:
def test(model, data_loader, device):
    # meters
    loss_meter = AverageMeter()
    acc_meter = AverageMeter()
    # switch to test mode
    correct = 0
    model.eval()
    tk = tqdm(data_loader, total=int(len(data_loader)), desc='Test', unit='frames', leave=False)
    for batch_idx, data in enumerate(tk):
        # fetch the data
        frame, label = data[0], data[1]
        frame = torch.squeeze(frame)
        # after fetching the data transfer the model to the 
        # required device, in this example the device is gpu
        # transfer to gpu can also be done by 
        frame, label = frame.to(device), label.to(device)
        # since we dont need to backpropagate loss in testing,
        # we dont keep the gradient
        with torch.no_grad():
            output = model(frame)
        
        logits = output.logits
        # compute the loss function just for checking
        loss_this = F.cross_entropy(logits, label)
        # get the index of the max log-probability
        pred = logits.argmax(dim=1, keepdim=True)
        # check which of the predictions are correct
        correct_this = pred.eq(label.view_as(pred)).sum().item()
        # accumulate the correct ones
        correct += correct_this
        # compute accuracy
        acc_this = correct_this / label.shape[0] * 100.0
        # update the loss and accuracy meter 
        acc_meter.update(acc_this, label.shape[0])
        loss_meter.update(loss_this.item(), label.shape[0])
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        loss_meter.avg, correct, len(data_loader.dataset), acc_meter.avg))

In [7]:
test(model,test_loader,device)

Test:   0%|          | 0/19 [00:00<?, ?frames/s]

Test: Average loss: 0.9385, Accuracy: 262/363 (72.18%)

